## Install required extensions

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         219 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0



beautifulsoup4-4.9.1 | 163 KB    | ##################################### | 100% 
soupsieve-2.0.1      | 56 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction

## Import required libraries

In [2]:
import urllib.request #import library to use to read URLs
import requests
from bs4 import BeautifulSoup
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from pandas.io.json import json_normalize # tranforming json file into a pandas dataframe library
from urllib.request import urlopen

## Scrap information from Wikipedia Website

In [3]:
#specify URL to scrap
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [8]:
soup = BeautifulSoup(url, "html.parser")

In [101]:
My_table = soup.find('table', class_='wikitable sortable')

In [37]:
A=[]
B=[]
C=[]

for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))


In [38]:
df = pd.DataFrame(A, columns=['PostalCode'])
df['Borough'] = B
df['Neighborhood'] = C
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [64]:
#remove \n from data
df_postal = df['PostalCode'].str.slice(0, -1)
df_borough = df['Borough'].str.slice(0, -1)
df_neighbor = df['Neighborhood'].str.slice(0, -1)

In [82]:
#Create new dataframe without '\n'
df_right = pd.DataFrame(df_clean, columns=['PostalCode'])
df_right['Borough'] = df_borough
df_right['Neighborhood'] = df_neighbor
df_right.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [85]:
#drop Borough with 'Not assigned'
df_right.drop(df_right[df_right.Borough == 'Not assigned'].index, inplace = True)

In [90]:
#make Neighborhood = Borough with Neighborhood 'Not assigned'
df_right.loc[df_right['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_right['Borough']

In [99]:
#Check if any repeated PostalCodes
boolean = df_right.duplicated(subset = ['PostalCode']).any()
boolean

False

In [100]:
#Check shape of data
df_right.shape

(103, 3)